In [1]:
import queue


def task(name, work_queue):
    if work_queue.empty():
        print(f"Task {name} nothing to do")

    else:
        while not work_queue.empty():
            count = work_queue.get()
            total = 0
            print(f"Task {name} running")
            for x in range(count):
                total += 1
            print(f"Task {name} total: {total}")


def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = queue.Queue()

    # Put some work in the queue
    for work in [15, 10, 5, 2]:
        work_queue.put(work)

    # Create some synchronous tasks
    tasks = [(task, "One", work_queue), (task, "Two", work_queue)]

    # Run the tasks
    for t, n, q in tasks:
        t(n, q)


if __name__ == "__main__":
    main()

Task One running
Task One total: 15
Task One running
Task One total: 10
Task One running
Task One total: 5
Task One running
Task One total: 2
Task Two nothing to do


In [7]:
import queue


def task(name, work_queue):
    while not work_queue.empty():
        count = work_queue.get()
        total = 0
        print(f"Task {name} running")
        for x in range(count):
            total += 1
            yield
        print(f"Task {name} total: {total}")


def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = queue.Queue()

    # Put some work in the queue
    for work in [15, 10, 5, 2]:
        work_queue.put(work)

    # Create some synchronous tasks
    tasks = [task("One", work_queue), task("Two", work_queue)]

    # Run the tasks
    done = False
    while not done:
        for t in tasks:
            try:
                next(t)
            except StopIteration:
                tasks.remove(t)
            if len(tasks) == 0:
                done = True


if __name__ == "__main__":
    main()

Task One running
Task Two running
Task Two total: 10
Task Two running
Task One total: 15
Task One running
Task Two total: 5
Task One total: 2


In [8]:
import time

import queue

from codetiming import Timer


def task(name, queue):
    timer = Timer(text=f"Task {name} elapsed time: {{:.1f}}")
    while not queue.empty():
        delay = queue.get()
        print(f"Task {name} running")
        timer.start()
        time.sleep(delay)
        timer.stop()
        yield


def main():
    """
    This is the main entry point for the program
    """

    # Create the queue of work
    work_queue = queue.Queue()
    # Put some work in the queue
    for work in [15, 10, 5, 2]:
        work_queue.put(work)
    tasks = [task("One", work_queue), task("Two", work_queue)]

    # Run the tasks
    done = False
    with Timer(text="\nTotal elapsed time: {:.1f}"):
        while not done:
            for t in tasks:
                try:
                    next(t)
                except StopIteration:
                    tasks.remove(t)
                if len(tasks) == 0:
                    done = True


if __name__ == "__main__":
    main()

Task One running
Task One elapsed time: 15.0
Task Two running
Task Two elapsed time: 10.0
Task One running
Task One elapsed time: 5.0
Task Two running
Task Two elapsed time: 2.0

Total elapsed time: 32.0


In [12]:
import asyncio
from codetiming import Timer


async def task(name, work_queue):
    timer = Timer(text=f"Task {name} elapsed time: {{:.1f}}")
    while not work_queue.empty():
        delay = await work_queue.get()
        print(f"Task {name} running")
        timer.start()
        await asyncio.sleep(delay)
        timer.stop()


async def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = asyncio.Queue()
    # Put some work in the queue

    for work in [15, 10, 5, 2]:
        await work_queue.put(work)

    # Run the tasks
    with Timer(text="\nTotal elapsed time: {:.1f}"):
        result = await asyncio.gather(
            asyncio.create_task(task("One", work_queue)),
            asyncio.create_task(task("Two", work_queue)),
        )
        print(result)


await main()

Task One running
Task Two running
Task Two elapsed time: 10.0
Task Two running
Task One elapsed time: 15.0
Task One running
Task Two elapsed time: 5.0
Task One elapsed time: 2.0
[None, None]

Total elapsed time: 17.0


In [13]:
import queue
import requests
from codetiming import Timer


def task(name, work_queue):
    timer = Timer(text=f"Task {name} elapsed time: {{:.1f}}")
    with requests.Session() as session:
        while not work_queue.empty():
            url = work_queue.get()
            print(f"Task {name} getting URL: {url}")
            timer.start()
            session.get(url)
            timer.stop()
            yield


def main():
    """
    This is the main entry point for the program
    """
    # Create the queue of work

    work_queue = queue.Queue()
    # Put some work in the queue
    for url in [
        "http://google.com",
        "http://yahoo.com",
        "http://linkedin.com",
        "http://apple.com",
        "http://microsoft.com",
        "http://facebook.com",
        "http://twitter.com",
    ]:
        work_queue.put(url)
    
    tasks = [task("One", work_queue), task("Two", work_queue)]

    # Run the tasks

    done = False
    with Timer(text="\nTotal elapsed time: {:.1f}"):

        while not done:
            for t in tasks:
                try:
                    next(t)
                except StopIteration:
                    tasks.remove(t)
                if len(tasks) == 0:
                    done = True


if __name__ == "__main__":
    main()

Task One getting URL: http://google.com
Task One elapsed time: 0.3
Task Two getting URL: http://yahoo.com
Task Two elapsed time: 1.7
Task One getting URL: http://linkedin.com
Task One elapsed time: 1.1
Task Two getting URL: http://apple.com
Task Two elapsed time: 0.6
Task One getting URL: http://microsoft.com
Task One elapsed time: 0.7
Task Two getting URL: http://facebook.com
Task Two elapsed time: 1.0
Task One getting URL: http://twitter.com
Task One elapsed time: 0.6

Total elapsed time: 6.1


In [4]:
import asyncio
import aiohttp
from codetiming import Timer


async def task(name, work_queue):
    timer = Timer(text=f"Task {name} elapsed time: {{:.1f}}")
    async with aiohttp.ClientSession() as session:
        while not work_queue.empty():
            url = await work_queue.get()
            print(f"Task {name} getting URL: {url}")
            
            timer.start()
            async with session.get(url) as response:
                await response.text()
            timer.stop()


async def main():

    """
    This is the main entry point for the program
    """
    # Create the queue of work
    work_queue = asyncio.Queue()
    # Put some work in the queue

    for url in [
        "http://google.com",
        "http://yahoo.com",
        "http://linkedin.com",
        "http://apple.com",
        "http://microsoft.com",
        "http://facebook.com",
        "http://twitter.com",

    ]:
        await work_queue.put(url)

    # Run the tasks
    with Timer(text="\nTotal elapsed time: {:.1f}"):

        result  = await asyncio.gather(
            asyncio.create_task(task("One", work_queue)),
            asyncio.create_task(task("Two", work_queue)),
            asyncio.create_task(task("Three", work_queue)),
            asyncio.create_task(task("Four", work_queue)),
        )
        print (result)



await main()

Task One getting URL: http://google.com
Task Two getting URL: http://yahoo.com
Task Three getting URL: http://linkedin.com
Task Four getting URL: http://apple.com
Task One elapsed time: 0.2
Task One getting URL: http://microsoft.com
Task Four elapsed time: 0.4
Task Four getting URL: http://facebook.com
Task Three elapsed time: 0.7
Task Three getting URL: http://twitter.com
Task Three elapsed time: 0.5
Task Four elapsed time: 0.8
Task Two elapsed time: 2.3
Task One elapsed time: 2.2
[None, None, None, None]

Total elapsed time: 2.5


this is same

In [24]:
import requests
array = [
        "http://google.com",
        "http://yahoo.com",
        "http://linkedin.com",
        "http://apple.com",
        "http://microsoft.com",
        "http://facebook.com",
        "http://twitter.com"]

def main():
    with Timer(text="\nTotal elapsed time: {:.1f}"):
        for url in array:
            response = requests.get(url)
            print(f"Response from {url}: {response.status_code}")
main()

Response from http://google.com: 200
Response from http://yahoo.com: 200
Response from http://linkedin.com: 200
Response from http://apple.com: 200
Response from http://microsoft.com: 200
Response from http://facebook.com: 200
Response from http://twitter.com: 400

Total elapsed time: 6.5


as this

In [22]:
import asyncio
import requests
from codetiming import Timer

array = [
        "http://google.com",
        "http://yahoo.com",
        "http://linkedin.com",
        "http://apple.com",
        "http://microsoft.com",
        "http://facebook.com",
        "http://twitter.com"]

async def get_site_async(url):
    response = requests.get(url)
    print(f"Response from {url}: {response.status_code}")

async def main():
    
    with Timer(text="\nTotal elapsed time: {:.1f}"):
         await asyncio.gather(
            *[asyncio.create_task(get_site_async(url)) for url in array]
        )
    
await main()

Response from http://google.com: 200
Response from http://yahoo.com: 200
Response from http://linkedin.com: 200
Response from http://apple.com: 200
Response from http://microsoft.com: 200
Response from http://facebook.com: 200
Response from http://twitter.com: 400

Total elapsed time: 6.7


In [5]:
array = [
        "http://google.com",
        "http://yahoo.com",
        "http://linkedin.com",
        "http://apple.com",
        "http://microsoft.com",
        "http://facebook.com",
        "http://twitter.com"]

import httpx
import asyncio
async with httpx.AsyncClient() as session:  #use httpx
    result = await asyncio.gather(*[session.get(url) for url in array])
    print(result)

[<Response [301 Moved Permanently]>, <Response [301 Moved Permanently]>, <Response [301 Moved Permanently]>, <Response [301 Redirect]>, <Response [301 Moved Permanently]>, <Response [301 Moved Permanently]>, <Response [301 Moved Permanently]>]


In [7]:
import requests
from requests import Response
import asyncio
from asyncio import Task

async def fetch_status(url:str) -> dict:
    print(f'Fetching status for : {url}')
    respons: Response = await asyncio.to_thread(requests.get, url, None)
    print('Done!')
    return {'status': respons.status_code , 'url': url}

async def main():
    taks1: Task[dict] = asyncio.create_task(fetch_status("http://google.com"))
    taks2: Task[dict] = asyncio.create_task(fetch_status("http://yahoo.com"))

    status1 = await taks1
    status2 = await taks2

    print(status1)
    print(status2)
    
await main()

Fetching status for : http://google.com
Fetching status for : http://yahoo.com
Done!
Done!
{'status': 200, 'url': 'http://google.com'}
{'status': 200, 'url': 'http://yahoo.com'}
